<a href="https://colab.research.google.com/github/m-manuelmussa/Desenvolvimento-de-modelos-de-machine-learning-para-predizer-novos-inibidores-da-TK-HER2/blob/main/Desenvolvimento_de_modelos_de_machine_learning_baseados_em_QSAR_3D_para_a_predi%C3%A7%C3%A3o_de_novos_candidatos_a_f%C3%A1rmacos_TKI_HER2_para_o_tratamento_de_c%C3%A2ncer_da_mama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Desenvolvimento de modelos de machine learning baseados em QSAR-3D para a predição de novos candidatos a fármacos TKI-HER2 para o tratamento de câncer da mama***

---



# ***1. Preparo do banco de dados***

# **Pipeline**
1. Configuração do ambiente de trabalho
2. Selecção do dataset de compostos inibidores da TK-HER2
3. Seleção dos compostos com dados de IC₅₀
4. Limpeza dos dados (sem duplicatas, IC₅₀ e SMILES ausentes)
5. Dataset final limpo e pronto para análise

### ***1. Configuração do ambiente de trabalho***

In [ ]:
#Instalação de frameworks necessários
!pip install chembl_webresource_client

In [ ]:
#Importação de frameworks necessários
from chembl_webresource_client.new_client import new_client
import pandas as pd

### ***2. Selecção do dataset de compostos inibidores da TK-HER2***

In [ ]:
# 1. Seleção do alvo HER2 (tirosina-quinase)
alvo_especifico = "CHEMBL1824"  # HER2/ERBB2

# 2. Buscar atividades biológicas relacionadas ao alvo
actividades = new_client.activity.filter(target_chembl_id=alvo_especifico).only(
    ["molecule_chembl_id"]
)

# 3. Converter o QuerySet para lista antes de criar o DataFrame
actividades_list = list(actividades)

# 4. Criar DataFrame
df = pd.DataFrame(actividades_list)

# 5. Contar compostos únicos testados contra HER2
total_compostos_unicos = df['molecule_chembl_id'].nunique()

# 6. Exibir o resultado
print(f"DATASET INICIAL: {total_compostos_unicos} COMPOSTOS")

DATASET INICIAL: 5577 COMPOSTOS


### ***3. Seleção dos compostos com dados de IC₅₀***

In [ ]:
# 1. Buscar atividades com IC50 contra HER2
atividades_ic50 = new_client.activity.filter(
    target_chembl_id=alvo_especifico,
    standard_type="IC50"
).only(["molecule_chembl_id", "standard_value"])

# 2. Contar compostos únicos com IC50
compostos_unicos = set()
for atividade in atividades_ic50:
    if atividade.get("standard_value") is not None:
        compostos_unicos.add(atividade["molecule_chembl_id"])

# 3. Exibir resultado
print(f"DATASET SELECCIONADO: {len(compostos_unicos)} COMPOSTOS")

DATASET SELECCIONADO: 2729 COMPOSTOS


### ***4. Limpeza dos dados (sem duplicatas, IC₅₀ e SMILES ausentes)***

In [31]:
# 1. Transformar em lista e DataFrame
atividades_list = list(atividades_ic50)
df = pd.DataFrame(atividades_list)

# 2. Remover dados sem IC50
df = df.dropna(subset=["standard_value"])
df["IC50_nM"] = df["standard_value"].astype(float)

# 3. Buscar os SMILES das moléculas
from tqdm import tqdm  # barra de progresso
tqdm.pandas()

molecule = new_client.molecule

def get_smiles(chembl_id):
    try:
        mol = molecule.get(chembl_id)
        if mol.get("molecule_structures"):
            return mol["molecule_structures"]["canonical_smiles"]
    except:
        return None

# 4. Adicionar coluna de SMILES
df["canonical_smiles"] = df["molecule_chembl_id"].progress_apply(get_smiles)

# 5. Remover compostos sem SMILES ou IC50
df_clean = df.dropna(subset=["canonical_smiles", "IC50_nM"])

# 6. Remover duplicatas por SMILES
df_clean = df_clean.drop_duplicates(subset="canonical_smiles")

100%|██████████| 3228/3228 [00:05<00:00, 589.04it/s]


### ***5. Dataset final limpo e pronto para análise***

In [32]:
# 1. Resultado final
df_clean.head()
print(f"\nDATASET FINAL: {len(df_clean)} COMPOSTOS")


DATASET FINAL: 2725 COMPOSTOS


In [33]:
# 2. Selecionar colunas desejadas e renomear
df_export = df[["molecule_chembl_id", "canonical_smiles", "IC50_nM"]].rename(columns={
    "molecule_chembl_id": "ChEMBL_ID",
    "canonical_smiles": "SMILES",
    "IC50_nM": "IC50_nM"
})

# 3. Exportar para CSV com separador ";"
df_export.to_csv("HER2_compostos_IC50.csv", sep=";", index=False)

print("Arquivo CSV salvo como 'HER2_compostos_IC50.csv'")


Arquivo CSV salvo como 'HER2_compostos_IC50.csv'
